# Data Cleaning and Preparation for EDA

**Goal:** Transform the raw, combined dataset from the Dugas (2009) and Chen (2011) dissertations into a clean, structured, and fully numerical format suitable for exploratory data analysis (EDA) and machine learning.

**Methodology:**
1.  Load the raw data from Google Drive.
2.  Inspect the data for missing values and inconsistencies.
3.  Engineer new numerical features from the `Amine` and `Concentration (m)` columns to handle blended solvents.
4.  Standardize column names and finalize the DataFrame.
5.  Save the cleaned data back to Google Drive for future use.

In [2]:
# Step 1: Initial Setup

# Import essential libraries for data manipulation and analysis
import pandas as pd
import numpy as np

# Mount your Google Drive to access project files
from google.colab import drive
drive.mount('/content/drive')

# Define the path to your project folder in Google Drive
# IMPORTANT: Make sure this path is correct for your Drive setup
PROJECT_PATH = '/content/drive/MyDrive/CO2-Capture-ML/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Step 2: Load and Inspect the Raw Data

We'll start by loading the dataset from the `data/raw` directory. The initial inspection using `.info()` and `.isnull().sum()` is crucial to understand the data types, identify missing values, and get a general sense of the dataset's structure.

In [4]:
# Define the full path to the raw CSV file
raw_file_path = '/content/drive/MyDrive/CO2-Capture-ML/Data/Raw/CO2 Absorption Dataset.csv' # Assuming you named it this

# Load the data into a pandas DataFrame
df = pd.read_csv(raw_file_path)

# Display the first 5 rows to see how the data looks
print("--- Initial Data Preview ---")
display(df.head())

# Get a summary of the DataFrame, including data types and non-null counts
print("\n--- Initial DataFrame Info ---")
df.info()

# Check for the total number of missing values in each column
print("\n--- Missing Values Per Column ---")
print(df.isnull().sum())

--- Initial Data Preview ---


,Amine,Concentration (m),Temperature (K),CO2​ Loading (mol/mol_alk),PCO2∗​ (Pa),Target -> kg′​ (mol/s·Pa·m²),Source
0,MEA,7,313.15,0.252,15.7,3.34E-06,Dugas 2009
1,MEA,7,313.15,0.351,77.0,1.40E-06,Dugas 2009
2,MEA,7,313.15,0.432,465.0,7.66E-07,Dugas 2009
3,MEA,7,313.15,0.496,4216.0,3.47E-07,Dugas 2009
4,MEA,7,333.15,0.252,109.0,2.92E-06,Dugas 2009



--- Initial DataFrame Info ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 293 entries, 0 to 292
Data columns (total 7 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Amine                         293 non-null    object 
 1   Concentration (m)             293 non-null    object 
 2   Temperature (K)               293 non-null    float64
 3   CO2​ Loading (mol/mol_alk)    293 non-null    float64
 4   PCO2∗​ (Pa)                   293 non-null    float64
 5   Target -> kg′​ (mol/s·Pa·m²)  293 non-null    object 
 6   Source                        293 non-null    object 
dtypes: float64(3), object(4)
memory usage: 16.2+ KB

--- Missing Values Per Column ---
Amine                           0
Concentration (m)               0
Temperature (K)                 0
CO2​ Loading (mol/mol_alk)      0
PCO2∗​ (Pa)                     0
Target -> kg′​ (mol/s·Pa·m²)    0
Source                          0
dtype: 

## Step 3: Handle Missing Values

Our inspection shows one missing value in the target column, `Target -> k_g' (mol/s·Pa·m²)`. Since this is only a single data point out of 185, the safest and most effective strategy is to remove the entire row. Imputing or guessing this value could introduce bias into our model later on.

In [5]:
print(f"Original shape of the DataFrame: {df.shape}")

# Drop rows with any missing values
df_cleaned = df.dropna().reset_index(drop=True)

print(f"Shape after dropping missing rows: {df_cleaned.shape}")

# Verify that no missing values remain
print("\n--- Missing values after cleaning ---")
print(df_cleaned.isnull().sum())

Original shape of the DataFrame: (293, 7)
Shape after dropping missing rows: (293, 7)

--- Missing values after cleaning ---
Amine                           0
Concentration (m)               0
Temperature (K)                 0
CO2​ Loading (mol/mol_alk)      0
PCO2∗​ (Pa)                     0
Target -> kg′​ (mol/s·Pa·m²)    0
Source                          0
dtype: int64


## Step 4: Feature Engineering for Blended Amines

This is the most critical preprocessing step. The `Amine` and `Concentration (m)` columns contain mixed data types (strings and numbers) which are not suitable for a machine learning model.

We will engineer new features by:
1.  Identifying every unique amine component in the dataset (e.g., MEA, PZ, 2MPZ).
2.  Creating a new numerical column for each unique amine (e.g., `Conc_MEA_(m)`).
3.  Parsing the original `Amine` and `Concentration (m)` columns to populate these new features with the correct molality values, using 0.0 for amines not present in a given solvent blend.

In [6]:
# Create a working copy to avoid modifying the cleaned DataFrame directly
df_eng = df_cleaned.copy()

# 1. Identify all unique amine components
unique_amines = set()
for amine_str in df_eng['Amine'].unique():
    amines = amine_str.split('/')
    for amine in amines:
        cleaned_amine = ''.join(filter(str.isalpha, amine))
        if cleaned_amine:
            unique_amines.add(cleaned_amine)

unique_amines = sorted(list(unique_amines))
print(f"Found {len(unique_amines)} unique amine components: {unique_amines}")

# 2. Create new columns for each amine, initialized to zero
for amine in unique_amines:
    df_eng[f'Conc_{amine}_(m)'] = 0.0

# 3. Populate the new columns by parsing the original data
for index, row in df_eng.iterrows():
    amine_components = row['Amine'].split('/')

    try:
        # For strings like '7m/2m', remove 'm' and split
        conc_str = row['Concentration (m)'].replace('m', '')
        concentrations = [float(c) for c in conc_str.split('/')]
    except AttributeError:
        # For numerical values
        concentrations = [float(row['Concentration (m)'])]

    for i, component in enumerate(amine_components):
        # Clean the component name (e.g., '7m' -> 'm', needs to be handled)
        # We rely on the `Amine` column for the name, not the concentration string
        cleaned_component = ''.join(filter(str.isalpha, amine_components[i]))
        if cleaned_component in unique_amines:
            df_eng.loc[index, f'Conc_{cleaned_component}_(m)'] = concentrations[i]

# Display the first few rows with the new columns to verify
print("\n--- DataFrame after Feature Engineering ---")
display(df_eng.head())

Found 12 unique amine components: ['AEP', 'AMP', 'DGA', 'DMPZ', 'EDA', 'HEP', 'MAPA', 'MDEA', 'MEA', 'MPZ', 'PE', 'PZ']

--- DataFrame after Feature Engineering ---


,Amine,Concentration (m),Temperature (K),CO2​ Loading (mol/mol_alk),PCO2∗​ (Pa),Target -> kg′​ (mol/s·Pa·m²),Source,Conc_AEP_(m),Conc_AMP_(m),Conc_DGA_(m),Conc_DMPZ_(m),Conc_EDA_(m),Conc_HEP_(m),Conc_MAPA_(m),Conc_MDEA_(m),Conc_MEA_(m),Conc_MPZ_(m),Conc_PE_(m),Conc_PZ_(m)
0,MEA,7,313.15,0.252,15.7,3.34E-06,Dugas 2009,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,0.0,0.0,0.0
1,MEA,7,313.15,0.351,77.0,1.40E-06,Dugas 2009,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,0.0,0.0,0.0
2,MEA,7,313.15,0.432,465.0,7.66E-07,Dugas 2009,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,0.0,0.0,0.0
3,MEA,7,313.15,0.496,4216.0,3.47E-07,Dugas 2009,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,0.0,0.0,0.0
4,MEA,7,333.15,0.252,109.0,2.92E-06,Dugas 2009,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,0.0,0.0,0.0


## Step 5: Finalizing the DataFrame

Now that we have created purely numerical features for the solvent composition, we can clean up the DataFrame. We will:
1.  Drop the original `Amine`, `Concentration (m)`, and `Source` columns.
2.  Standardize all column names to a consistent, code-friendly format (snake_case).
3.  Perform a final check for duplicates and data types.

In [7]:
# 1. Drop the original and metadata columns
df_final = df_eng.drop(columns=['Amine', 'Concentration (m)', 'Source'])

# 2. Standardize column names
# This makes them easier to call, e.g., df.co2_loading instead of df['CO2 Loading (...)']
new_column_names = {
    'Temperature (K)': 'temperature_k',
    'CO2 Loading (mol/mol_alk)': 'co2_loading_mol_per_mol_alk',
    'P*_CO2 (Pa)': 'p_star_co2_pa',
    "Target -> k_g' (mol/s·Pa·m²)": 'target_kg_prime',
}
# Rename the concentration columns as well
for amine in unique_amines:
    new_column_names[f'Conc_{amine}_(m)'] = f'conc_{amine.lower()}_m'

df_final = df_final.rename(columns=new_column_names)

# --- Final Verification ---
print("--- Final DataFrame Info ---")
df_final.info()

print("\n--- Final DataFrame Preview ---")
display(df_final.head())

# Check for any duplicate rows
print(f"\nNumber of duplicate rows: {df_final.duplicated().sum()}")

--- Final DataFrame Info ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 293 entries, 0 to 292
Data columns (total 16 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   temperature_k                 293 non-null    float64
 1   CO2​ Loading (mol/mol_alk)    293 non-null    float64
 2   PCO2∗​ (Pa)                   293 non-null    float64
 3   Target -> kg′​ (mol/s·Pa·m²)  293 non-null    object 
 4   conc_aep_m                    293 non-null    float64
 5   conc_amp_m                    293 non-null    float64
 6   conc_dga_m                    293 non-null    float64
 7   conc_dmpz_m                   293 non-null    float64
 8   conc_eda_m                    293 non-null    float64
 9   conc_hep_m                    293 non-null    float64
 10  conc_mapa_m                   293 non-null    float64
 11  conc_mdea_m                   293 non-null    float64
 12  conc_mea_m                    293 n

,temperature_k,CO2​ Loading (mol/mol_alk),PCO2∗​ (Pa),Target -> kg′​ (mol/s·Pa·m²),conc_aep_m,conc_amp_m,conc_dga_m,conc_dmpz_m,conc_eda_m,conc_hep_m,conc_mapa_m,conc_mdea_m,conc_mea_m,conc_mpz_m,conc_pe_m,conc_pz_m
0,313.15,0.252,15.7,3.34E-06,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,0.0,0.0,0.0
1,313.15,0.351,77.0,1.40E-06,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,0.0,0.0,0.0
2,313.15,0.432,465.0,7.66E-07,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,0.0,0.0,0.0
3,313.15,0.496,4216.0,3.47E-07,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,0.0,0.0,0.0
4,333.15,0.252,109.0,2.92E-06,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,0.0,0.0,0.0



Number of duplicate rows: 0


In [8]:
# The target column name after your previous renaming step should be 'target_kg_prime'
# If you haven't renamed it yet, use the original name.
target_col_name = 'Target -> kg′​ (mol/s·Pa·m²)' # Use this if you haven't run the renaming code yet
# target_col_name = 'target_kg_prime' # Use this if you have already run the renaming code

# Convert the target column from object (text) to a numerical type (float)
# The 'errors=coerce' argument will turn any values that can't be converted into NaN (missing)
df_final[target_col_name] = pd.to_numeric(df_final[target_col_name], errors='coerce')

# It's good practice to check if any errors occurred during conversion
print("--- Check for conversion errors (NaNs) ---")
print(df_final[target_col_name].isnull().sum())

# Now, let's look at the .info() again. The target column should now be float64.
print("\n--- Final Info After All Cleaning ---")
df_final.info()

--- Check for conversion errors (NaNs) ---
3

--- Final Info After All Cleaning ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 293 entries, 0 to 292
Data columns (total 16 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   temperature_k                 293 non-null    float64
 1   CO2​ Loading (mol/mol_alk)    293 non-null    float64
 2   PCO2∗​ (Pa)                   293 non-null    float64
 3   Target -> kg′​ (mol/s·Pa·m²)  290 non-null    float64
 4   conc_aep_m                    293 non-null    float64
 5   conc_amp_m                    293 non-null    float64
 6   conc_dga_m                    293 non-null    float64
 7   conc_dmpz_m                   293 non-null    float64
 8   conc_eda_m                    293 non-null    float64
 9   conc_hep_m                    293 non-null    float64
 10  conc_mapa_m                   293 non-null    float64
 11  conc_mdea_m                   293 non-n

## Step 6: Saving the Processed Data

The data is now clean, fully numerical, and ready for analysis. We will save this final DataFrame to the `data/processed` directory. This is a critical step for a reproducible workflow, as it creates a clear checkpoint. All future analysis and modeling should use this cleaned file.

In [11]:
# Define the path for the cleaned data file
processed_file_path = '/content/drive/MyDrive/CO2-Capture-ML/Data/Processed/Cleaned_Data.csv'

# Save the final DataFrame to a new CSV file without the pandas index
df_final.to_csv(processed_file_path, index=False)

print(f"✅ Cleaned data successfully saved to: {processed_file_path}")

✅ Cleaned data successfully saved to: /content/drive/MyDrive/CO2-Capture-ML/Data/Processed/Cleaned_Data.csv
